In [176]:
# "predict median for that building&meter, for that day of week"

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("../input/ashrae-energy-prediction/train.csv")


In [177]:
# add day of week and day of year

df["timestamp"] = pd.to_datetime(train["timestamp"])
df['dayofweek'] = df["timestamp"].dt.dayofweek
df['dayofyear'] = df["timestamp"].dt.dayofyear

df.sample(n=10, random_state=42)

,building_id,meter,timestamp,meter_reading,dayofweek,dayofyear
14245562,1324,1,2016-09-16 16:00:00,0.0000,4,260
1282718,1013,0,2016-01-24 06:00:00,32.0001,6,24
13883790,229,1,2016-09-10 07:00:00,567.6550,5,254
4781820,217,3,2016-04-01 01:00:00,0.0000,4,92
10415393,1434,0,2016-07-10 04:00:00,65.7500,6,192
1057008,1047,0,2016-01-20 04:00:00,90.9833,2,20
4507399,911,1,2016-03-26 20:00:00,295.0640,5,86
19478829,1039,0,2016-12-18 23:00:00,16.9000,6,353
8955615,265,0,2016-06-14 06:00:00,128.3700,1,166
13799839,896,0,2016-09-08 19:00:00,300.0000,3,252


In [184]:
# sum each day for meters 
# (note we might want to look at imputing missing values)

df_sum = df.groupby(['building_id','meter','dayofyear','dayofweek',]).sum()
df_sum.reset_index(inplace=True)
df_sum.loc[df_sum['building_id'] == 1047]

,building_id,meter,dayofyear,dayofweek,meter_reading
543112,1047,0,1,4,2272.2535
543113,1047,0,2,5,4853.8555
543114,1047,0,3,6,5005.9532
543115,1047,0,4,0,4581.4444
543116,1047,0,5,1,3687.0178
543117,1047,0,6,2,3923.4404
543118,1047,0,7,3,3073.0035
543119,1047,0,8,4,2582.7300
543120,1047,0,9,5,3397.1142
543121,1047,0,10,6,2541.9870


In [191]:
# for each day for of the week find the median value for the year

df_med = df_sum.groupby(['building_id','meter','dayofweek']).median()
df_med.reset_index(inplace=True)
df_med.drop(['dayofyear'], axis=1, inplace=True)


df_med.loc[df_med['building_id'] == 1047] 

,building_id,meter,dayofweek,meter_reading
10577,1047,0,0,4266.57800
10578,1047,0,1,4155.65280
10579,1047,0,2,4240.38520
10580,1047,0,3,4284.91135
10581,1047,0,4,4088.15200
10582,1047,0,5,3115.20210
10583,1047,0,6,3096.51975


AttributeError: 'DataFrame' object has no attribute 'get_dummies'

In [193]:
# Split fit and predit
X_train, X_test, y_train, y_test = train_test_split(
    df_med.drop('meter_reading', axis=1), df_med['meter_reading'], 
    test_size=0.33, random_state=42)

dummy_mean = DummyRegressor(strategy='mean')

reg = LinearRegression().fit(X_train, y_train)
predict_median = reg.predict(X_test)
print(predict_median)

# calcuate error, which looks really high
mean_absolute_error(y_test, predict_median)

[ 2224.15942146  4732.53964449  5160.0135883  ...  4145.33859132
 13482.45804061  6744.70421004]


10294.006557998733